In [324]:
from sympy import *
from sympy import init_printing
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.optimize import newton

init_printing(use_unicode=False, wrap_line=False, no_global=True)

In [325]:
def calculate_helmholtz_excess(
    rho_value,     
    reduced_temperature, 
    rho_inf, 
    eos
    ):
    # Defining symbols
    R, T, v, b, a, alpha, rho = symbols("R T v b a alpha rho")
    M, k = symbols("M xi")
    P_r, rho_r, T_r = symbols("P_r rho_r T_r")
    P_c, rho_c, T_c = symbols("P_c rho_c T_c")
    omega_a, omega_b = symbols("Omega_a Omega_b")
    Z_c = symbols("Z_c")
    rho_oo = symbols("rho_oo")
    
    P1 = eos.pressure.subs(v, M/rho)
    P2 = P1.subs(rho, rho_c * rho_r)
    P3 = P2.subs(T, T_c * T_r)
    P4 = P3/P_c
    P5 = P4.subs(a, omega_a*(R**2)*(T_c**2)/P_c)
    P6 = P5.subs(b, omega_b*R*T_c/P_c)
    P7 = expand(P6)
    P8 = P7.subs(T_c, P_c * M / (Z_c * rho_c * R)) 
    
    P_r = simplify(P8)    
    P_oo = P_r.subs(rho_r, rho_oo)    
    integrand = ((P_r - P_oo)/(rho_r ** 2))

    integrand = integrand.subs(omega_a, eos.omega_a)
    integrand = integrand.subs(omega_b, eos.omega_b)
    integrand = integrand.subs(Z_c, eos.critical_z_factor)
    integrand = integrand.subs(T_r, reduced_temperature)
    integrand = integrand.subs(rho_oo, rho_inf)
    
    func = lambdify(rho_r, integrand)
    y, err = integrate.quad(func, rho_inf, rho_value)
    return rho_value * y

In [326]:
# Defining symbols
P, R, T, v, b, a, alpha, rho = symbols("P R T v b a alpha rho")
def find_correct_root_of_cubic_eos(p0, p1, p2, p3, fluid_type):
    coef_a = (3.0 * p2 - (p1 ** 2)) / 3.0        
    coef_b = (2.0 * (p1 ** 3) - 9.0 * p1 * p2 + 27.0 * p3) / 27.0        
    delta = 0.25 * (coef_b ** 2) + (coef_a ** 3) / 27.0     

    if delta > 0.0:
        # 1 real root, 2 imaginary                 
        const_A =  cbrt(-0.5 * coef_b + sqrt(delta)) 
        const_B =  cbrt(-0.5 * coef_b - sqrt(delta))

        correct_root = const_A + const_B - p1 / 3.0 
    else:
        # 3 real roots
        phi = acos(-0.5 * coef_b / sqrt(-(coef_a ** 3) / 27.0))
        root_1 = 2.0 * sqrt(-coef_a / 3.0) * cos(phi / 3.0) - p1 / 3.0
        root_2 = 2.0 * sqrt(-coef_a / 3.0) * cos(phi / 3.0 + 2.0 * np.pi / 3.0) - p1 / 3.0
        root_3 = 2.0 * sqrt(-coef_a / 3.0) * cos(phi / 3.0 + 4.0 * np.pi / 3.0) - p1 / 3.0

        smallest_root = min(min(root_1,root_2), root_3)
        largest_root = max(max(root_1,root_2), root_3)

        if fluid_type is 'liquid':        
            correct_root = smallest_root
        else:
            assert fluid_type is 'vapor', 'Wrong fluid type! ' + fluid_type
            correct_root = largest_root
    
    assert correct_root > 0.0, fluid_type + ' Z-factor < 0.0! Delta is %f, %f' % (delta, correct_root)
    
    return correct_root

GAS_CONST = 8.314

class EquationOfState(object):
    def __init__(
        self,
        critical_pressure, 
        critical_temperature,
        acentric_factor,
        molar_mass
    ):       
        self.critical_pressure = critical_pressure
        self.critical_temperature = critical_temperature
        self.acentric_factor = acentric_factor
        self.molar_mass = molar_mass
        
        
    def calculate_pressure_and_densities(self, temperature):
        
        def func(x):            
            pressure = x
            Z_V = eos.calculate_eos_roots(pressure, temperature, 'vapor')
            Z_L = eos.calculate_eos_roots(pressure, temperature, 'liquid')
        
            f_V = eos.calculate_fugacity(pressure, temperature, Z_V)
            f_L = eos.calculate_fugacity(pressure, temperature, Z_L)
            
            return f_V - f_L
        
        pressure = newton(func=func, x0=1.0e5)
        
        Z_V = eos.calculate_eos_roots(pressure, temperature, 'vapor')
        Z_L = eos.calculate_eos_roots(pressure, temperature, 'liquid')
        
        rho_V = pressure * self.molar_mass / (Z_V * GAS_CONST * temperature)
        rho_L = pressure * self.molar_mass / (Z_L * GAS_CONST * temperature)
        
        return pressure, rho_V, rho_L
    
    
class PengRobinsonEos(EquationOfState):
    def __init__(
        self,
        critical_pressure, 
        critical_temperature,
        acentric_factor,
        molar_mass
    ):
        self.omega_a = 0.45724
        self.omega_b = 0.07780
        self.critical_z_factor = 0.3074
        self.pressure = R * T / (v - b) - a*alpha / (v*(v + b) + b*(v - b))
        
        EquationOfState.__init__(self, critical_pressure, critical_temperature, acentric_factor, molar_mass)

    
    def alpha_function(self, reduced_temperature):
        omega = self.acentric_factor
        
        if omega < 0.49:
            m = 0.37464 + 1.54226 * omega - 0.26992 * (omega ** 2)
        else:
            m = 0.379642 + 1.48503 * omega - 0.164423 * (omega ** 2) + 0.016667 * (omega ** 3)
        
        return (1.0 + m * (1.0 - sqrt(reduced_temperature))) ** 2
    
    
    def calculate_fugacity(
        self,
        pressure,
        temperature,
        z_factor
    ):
        P = pressure
        T = temperature
        Z = z_factor
        
        Pr = P / self.critical_pressure
        Tr = T / self.critical_temperature
        
        aa = self.alpha_function(Tr)
        
        A = aa * self.omega_a * Pr / (Tr ** 2)
        B = self.omega_b * Pr / Tr
        
        SQRT_2 = sqrt(2.0)        
        ln_f = (Z - 1.0) - log( Z - B ) - (A / (2.0 * SQRT_2 * B)) \
             * log( (Z + (1.0 + SQRT_2)*B)/(Z + (1.0 - SQRT_2)*B) )
        return P * exp(ln_f)
        
        
    def calculate_eos_roots(self, pressure, temperature, fluid_type):
        P = pressure
        T = temperature

        Pr = P / self.critical_pressure
        Tr = T / self.critical_temperature
        
        aa = self.alpha_function(Tr)
        
        A = aa * self.omega_a * Pr / (Tr ** 2)
        B = self.omega_b * Pr / Tr      
        
        p0 = 1.0
        p1 = - (1.0 - B)
        p2 = A - 3.0 * (B ** 2) - 2.0 * B
        p3 = -(A * B - B ** 2 - B ** 3)     

        return find_correct_root_of_cubic_eos(p0, p1, p2, p3, fluid_type)       
        
        
class VanDerWaalsEos(EquationOfState):
    def __init__(
        self,
        critical_pressure, 
        critical_temperature,
        acentric_factor,
        molar_mass
    ):
        self.omega_a = 27.0/64.0
        self.omega_b = 1.0/8.0
        self.critical_z_factor = 3.0/8.0
        self.pressure = R * T / (v - b) - a / (v**2)
       
        EquationOfState.__init__(self, critical_pressure, critical_temperature, acentric_factor, molar_mass)
        
    def calculate_fugacity(
        self,
        pressure,
        temperature,
        z_factor
    ):
        P = pressure
        T = temperature
        Z = z_factor
        Pr = P / self.critical_pressure
        Tr = T / self.critical_temperature
        
        A = self.omega_a * Pr / (Tr ** 2)
        B = self.omega_b * Pr / Tr
        
        ln_f = B / (Z - B) - log(Z - B) - 2.0 * A / Z
        return P * exp(ln_f)
    
    
    def calculate_eos_roots(self, pressure, temperature, fluid_type):
        P = pressure
        T = temperature

        Pr = P / self.critical_pressure
        Tr = T / self.critical_temperature
        
        A = self.omega_a * Pr / (Tr ** 2)
        B = self.omega_b * Pr / Tr
        
        p0 = 1.0
        p1 = - (B + 1.0)
        p2 = A
        p3 = - A * B   
        
        return find_correct_root_of_cubic_eos(p0, p1, p2, p3, fluid_type) 

In [327]:
temperature = 303.15
#pressure = 500.0 * 6894.75729
pressure = 1.0 * 1.0e5

# n-Butane properties
critical_pressure = 6894.75729 * 550.7
critical_temperature = (5.0 / 9.0) * 765.3 # [K]
acentric_factor = 0.192800 # [-]
molar_mass = 58.12 * 0.001 # [kg/mol]

eos = VanDerWaalsEos(critical_pressure, critical_temperature, acentric_factor, molar_mass)
Zfactor = eos.calculate_eos_roots(pressure, temperature, 'vapor')
print Zfactor
print eos.calculate_fugacity(pressure, temperature, Zfactor)


print 'newton results'


pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)
Z_V = eos.calculate_eos_roots(pressure, temperature, 'vapor')
Z_L = eos.calculate_eos_roots(pressure, temperature, 'liquid')
f_V = eos.calculate_fugacity(pressure, temperature, Z_V)
f_L = eos.calculate_fugacity(pressure, temperature, Z_L)

print f_V, f_L, rho_V, rho_L, pressure

0.982476817228185
98277.1196134455
newton results
714812.851555774 714812.851555785 23.2593689522383 352.245583099088 835569.339164810


# Step by step

1. Given the temperature and an Equation of State
2. Calculate the pressure and the densities

In [22]:
# n-Butane properties
critical_pressure = 6894.75729 * 550.7
critical_temperature = (5.0 / 9.0) * 765.3 # [K]
acentric_factor = 0.192800 # [-]
molar_mass = 58.12 * 0.001 # [kg/mol]

eos = VanDerWaalsEos(critical_pressure, critical_temperature, acentric_factor, molar_mass)

reduced_temperatures = np.linspace(0.4, 0.5, num=5)
#temperature = 303.15
for reduced_temperature in reduced_temperatures:
    temperature = reduced_temperature * critical_temperature
    
    pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)

    rho_c = eos.critical_pressure * eos.molar_mass / (eos.critical_z_factor * GAS_CONST * eos.critical_temperature)
    rho_inf = rho_L / rho_c 
    
    print reduced_temperature, rho_inf
    
    reduced_densities = np.linspace(0.01, 2.5, num=20)
    psi_vec = []
    for rho in reduced_densities:
        psi = calculate_helmholtz_excess(
            rho_value=rho, 
            reduced_temperature=reduced_temperature, 
            rho_inf=rho_inf,
            eos=eos
            )
        psi_vec.append(psi)
    psi_vec = np.array(psi_vec)

    plt.plot(reduced_densities, psi_vec, marker='o')


ax = plt.gca()
ax.grid(True)
plt.show()


0.4 2.58793748432490
0.425 2.55695590908282
0.45 2.52509676795955
0.475 2.49229809695853
0.5 2.45849200035004


In [23]:
# n-Butane properties
critical_pressure = 6894.75729 * 550.7
critical_temperature = (5.0 / 9.0) * 765.3 # [K]
acentric_factor = 0.192800 # [-]
molar_mass = 58.12 * 0.001 # [kg/mol]

eos = PengRobinsonEos(critical_pressure, critical_temperature, acentric_factor, molar_mass)

reduced_temperatures = np.linspace(0.8, 0.8, num=1)
#temperature = 303.15
for reduced_temperature in reduced_temperatures:
    temperature = reduced_temperature * critical_temperature    
    pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)

    rho_c = eos.critical_pressure * eos.molar_mass / (eos.critical_z_factor * GAS_CONST * eos.critical_temperature)
    rho_inf = rho_V / rho_c 
    
    print reduced_temperature, rho_inf, pressure, rho_V, rho_L
    
    eos.pressure = eos.pressure.subs(alpha, eos.alpha_function(reduced_temperature))    
    
    reduced_densities = np.linspace(0.1, 2.6, num=40)
    
    psi_vec = []
    reduced_densities_to_plot = []
    
    for rho in reduced_densities:
        psi = calculate_helmholtz_excess(
            rho_value=rho, 
            reduced_temperature=reduced_temperature, 
            rho_inf=rho_inf,
            eos=eos
            )
        
        if psi < 0.0:
            continue
            
        reduced_densities_to_plot.append(rho)
        psi_vec.append(psi)
        
    psi_vec = np.array(psi_vec)

    plt.plot(reduced_densities_to_plot, psi_vec, marker='o')


ax = plt.gca()
ax.grid(True)
plt.show()


0.8 0.0921619264335243 761687.713668340 18.7170808941358 534.554641682232


In [114]:
def gamma(kappa, eos, integral): 
    rho_c = eos.critical_pressure * eos.molar_mass / (eos.critical_z_factor * GAS_CONST * eos.critical_temperature)
    return rho_c * sqrt(eos.critical_pressure * kappa) * integral

def integrate_helholtz(eos, reduced_temperature, rho_0_r, rho_1_r, rho_inf=rho_V_r):
    def func(x):
        psi = calculate_helmholtz_excess(
                rho_value=x, 
                reduced_temperature=reduced_temperature, 
                rho_inf=rho_inf,
                eos=eos
                )
        return psi
        
    y, err = integrate.quad(func, rho_0_r, rho_1_r)
    return sqrt(2.0 * y)

# Calculate Interfacial Tension

In [116]:
reduced_temperature = 0.8
rho_V = 18.7170808941358 
rho_L = 534.554641682232

# n-Butane properties
critical_pressure = 6894.75729 * 550.7
critical_temperature = (5.0 / 9.0) * 765.3 # [K]
acentric_factor = 0.192800 # [-]
molar_mass = 58.12 * 0.001 # [kg/mol]

eos = PengRobinsonEos(critical_pressure, critical_temperature, acentric_factor, molar_mass)
eos.pressure = eos.pressure.subs(alpha, eos.alpha_function(reduced_temperature))  

rho_c = eos.critical_pressure * eos.molar_mass / (eos.critical_z_factor * GAS_CONST * eos.critical_temperature)

rho_V_r = rho_V / rho_c
rho_L_r = rho_L / rho_c

integral = integrate_helholtz(eos, reduced_temperature, rho_V_r, rho_L_r)

kappa_values = np.linspace(4.0e-17, 2.0e-16, num=20)

gamma_calc = []
for kappa in kappa_values:
    gamma_calc.append(gamma_VL(kappa, eos, integral))

    
plt.semilogy(gamma_calc, kappa_values, marker='o')


#plt.plot(gamma_calc, kappa_values, marker='o')
ax = plt.gca()
ax.grid(True, which=u'both')

plt.show()

# TODO

Calculate the above curve for various temperatures

Calculate the equilibrium contact angle as a function of $\phi^*$

$$
\cos \theta = \frac{\gamma_{sv} - \gamma_{sl}}{\gamma_{lv}}
$$

$$
\phi^* = \frac{\phi}{\sqrt{\kappa P_c}} = \pm \sqrt{2 \Psi^*_{exc}(\rho_s, T)}
$$

$$
\gamma_{vs} = \Phi(\rho_s) + \rho_c \sqrt{\kappa P_c} \int^{\rho^*_{sv}}_{\rho^*_{v}} \sqrt{2 \Psi^*_{exc}} d\rho^*
$$

$$
\gamma_{ls} = \Phi(\rho_s) + \rho_c \sqrt{\kappa P_c} \int^{\rho^*_{sl}}_{\rho^*_{l}} \sqrt{2 \Psi^*_{exc}} d\rho^*
$$

$$
\Phi(\rho_s) = - \phi \rho^*_{s} \rho_c
$$

$$
\phi = \phi^* \sqrt{\kappa P_c}
$$

Obs: Check out fig 3.9 to find the roots of eq $\phi^* = \frac{\phi}{\sqrt{\kappa P_c}} = \pm \sqrt{2 \Psi^*_{exc}(\rho_s, T)}$

We also can get $\kappa$ from 

$$
\gamma_{lv} = \rho_c \sqrt{2 \kappa P_c} \int^{\rho^*_{l}}_{\rho^*_{v}} \sqrt{\Psi^*_{exc}} d\rho^*
$$

First, let's plot the curve $\sqrt{2 \Psi^*_{exc}(\rho, T)}$ and intersect it with $\phi^* = 0.2$

In [265]:
psi_vec = {}
reduced_densities_to_plot = {}

In [398]:
reduced_temperature = 0.8

temperature = reduced_temperature * critical_temperature
pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)
print 'Tr, rho_V, rho_L', reduced_temperature, rho_V, rho_L


#reduced_temperature = 0.8
#rho_V = 18.7170808941358 
#rho_L = 534.554641682232

# n-Butane properties
critical_pressure = 6894.75729 * 550.7
critical_temperature = (5.0 / 9.0) * 765.3 # [K]
acentric_factor = 0.192800 # [-]
molar_mass = 58.12 * 0.001 # [kg/mol]

eos = PengRobinsonEos(critical_pressure, critical_temperature, acentric_factor, molar_mass)
eos.pressure = eos.pressure.subs(alpha, eos.alpha_function(reduced_temperature))  

rho_c = eos.critical_pressure * eos.molar_mass / (eos.critical_z_factor * GAS_CONST * eos.critical_temperature)

rho_V_r = rho_V / rho_c
rho_L_r = rho_L / rho_c

reduced_densities = np.linspace(0.0001, 3.5, num=150)
    
psi_vec[reduced_temperature] = []
reduced_densities_to_plot[reduced_temperature] = []

for rho in reduced_densities:
    psi = calculate_helmholtz_excess(
        rho_value=rho, 
        reduced_temperature=reduced_temperature, 
        rho_inf=rho_V_r,
        eos=eos
        )

    reduced_densities_to_plot[reduced_temperature].append(rho)
    psi_vec[reduced_temperature].append(sqrt(2.0*psi))

psi_vec[reduced_temperature] = np.array(psi_vec[reduced_temperature])

plt.plot(reduced_densities_to_plot[reduced_temperature], -psi_vec[reduced_temperature], marker='o')

phi_r = -0.275 * np.ones(len(reduced_densities_to_plot[reduced_temperature]))
plt.plot(reduced_densities_to_plot[reduced_temperature], phi_r, 'k--')

ax = plt.gca()
ax.grid(True)
plt.show()

Tr, rho_V, rho_L 0.8 18.7170808941358 534.554641682232


Now we need to find the roots of the equation $\phi^* - \sqrt{2 \Psi^*_{exc}(\rho, T)} = 0$ and find $\rho_s$. So let's define a residual equation as $R(\rho) = \phi^* - \sqrt{2 \Psi^*_{exc}(\rho, T)}$.


In [417]:
def calculate_surface_densities(phi_r, reduced_temperature, rho_V_r, rho_L_r, rho_inf):
    def func(x):
        psi = calculate_helmholtz_excess(
            rho_value=x, 
            reduced_temperature=reduced_temperature, 
            rho_inf=rho_inf,
            eos=eos
            )
        
        if phi_r > 0.0:
            return phi_r - sqrt(2.0 * psi)
        else:
            return phi_r + sqrt(2.0 * psi)

    if phi_r > 0.0:
        rho_SV_r = newton(func=func, x0=rho_V_r + 0.1*rho_V_r)
        rho_SL_r = newton(func=func, x0=rho_L_r + 0.1*rho_L_r)
    else:       
        from scipy.optimize import brentq
        rho_SV_r = brentq(f=func, a=0.000001, b=0.9999*rho_V_r)
        rho_SL_r = brentq(f=func, a=0.5*(rho_L_r + rho_V_r), b=0.9999*rho_L_r)
    
    return rho_SV_r, rho_SL_r

Now we need
$$
\gamma_{vs} = \Phi(\rho_s) + \rho_c \sqrt{\kappa P_c} \int^{\rho^*_{sv}}_{\rho^*_{v}} \sqrt{2 \Psi^*_{exc}} d\rho^*
$$

$$
\gamma_{ls} = \Phi(\rho_s) + \rho_c \sqrt{\kappa P_c} \int^{\rho^*_{sl}}_{\rho^*_{l}} \sqrt{2 \Psi^*_{exc}} d\rho^*
$$

$$
\Phi(\rho_s) = - \phi \rho^*_{s} \rho_c = - \rho^*_{s} \rho_c \phi^* \sqrt{\kappa P_c}
$$
$$
\gamma_{lv} = \rho_c \sqrt{2 \kappa P_c} \int^{\rho^*_{l}}_{\rho^*_{v}} \sqrt{\Psi^*_{exc}} d\rho^*
$$
$$
\cos \theta = \frac{\gamma_{sv} - \gamma_{sl}}{\gamma_{lv}}
$$


$$
\gamma_{vs}^* = \frac{\gamma_{vs}}{\rho_c\sqrt{\kappa P_c}} = - \rho^*_{s} \phi^* + \int^{\rho^*_{sv}}_{\rho^*_{v}} \sqrt{2 \Psi^*_{exc}} d\rho^*
$$

$$
\gamma_{ls}^* = \frac{\gamma_{ls}}{\rho_c\sqrt{\kappa P_c}} =- \rho^*_{s} \phi^* + \int^{\rho^*_{sl}}_{\rho^*_{l}} \sqrt{2 \Psi^*_{exc}} d\rho^*
$$

and

$$
\gamma_{lv}^* = \frac{\gamma_{lv}}{\rho_c\sqrt{\kappa P_c}} = \int^{\rho^*_{l}}_{\rho^*_{v}} \sqrt{2 \Psi^*_{exc}} d\rho^*
$$

$$
\cos \theta = \frac{\gamma_{sv} - \gamma_{sl}}{\gamma_{lv}} = \frac{\gamma_{vs}^* - \gamma_{ls}^*}{\gamma_{lv}^*}
$$

In [374]:
def calculate_contact_angle(phi_r, gamma_VL_r, eos, reduced_temperature, rho_V_r, rho_L_r):
    if abs(phi_r) < 1.0e-8:
        contact_angle = 90.0
    else:
        rho_SV_r, rho_SL_r = calculate_surface_densities(
            phi_r=phi_r, 
            reduced_temperature=reduced_temperature, 
            rho_V_r=rho_V_r, 
            rho_L_r=rho_L_r, 
            rho_inf=rho_V_r
        )

        if phi_r > 0.0:
            integral_SV = integrate_helholtz(eos, reduced_temperature, rho_V_r, rho_SV_r, rho_inf=rho_V_r)
            integral_SL = integrate_helholtz(eos, reduced_temperature, rho_L_r, rho_SL_r, rho_inf=rho_V_r)
        else:
            integral_SV = integrate_helholtz(eos, reduced_temperature, rho_SV_r, rho_V_r, rho_inf=rho_V_r)
            integral_SL = integrate_helholtz(eos, reduced_temperature, rho_SL_r, rho_L_r, rho_inf=rho_V_r)    

        gamma_SV_r = integral_SV - phi_r * rho_SV_r
        gamma_SL_r = integral_SL - phi_r * rho_SL_r

        contact_angle = acos((gamma_SV_r - gamma_SL_r) / gamma_VL_r) * 180.0 / np.pi
    
    return contact_angle

In [375]:
contact_angles = {}
phi_values = {}
phi_limits = {}
# Phi limits (so functions do not diverge)
# Tr = 0.9 -> phi_r = 0.35
# Tr = 0.8 -> phi_r = 0.275
# Tr = 0.7 -> phi_r = 0.238
# Tr = 0.65 -> phi_r = 0.225
phi_limits[0.9] = 0.35
phi_limits[0.8] = 0.275
phi_limits[0.7] = 0.238
phi_limits[0.65] = 0.225

In [436]:
# n-Butane properties
critical_pressure = 6894.75729 * 550.7
critical_temperature = (5.0 / 9.0) * 765.3 # [K]
acentric_factor = 0.192800 # [-]
molar_mass = 58.12 * 0.001 # [kg/mol]


reduced_temperature_values = [0.65, 0.7, 0.8, 0.9]
#reduced_temperature_values = [0.8]
for reduced_temperature in reduced_temperature_values:
    temperature = reduced_temperature * critical_temperature
    eos = PengRobinsonEos(critical_pressure, critical_temperature, acentric_factor, molar_mass)
    rho_c = eos.critical_pressure * eos.molar_mass / (eos.critical_z_factor * GAS_CONST * eos.critical_temperature)

    pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)
    eos.pressure = eos.pressure.subs(alpha, eos.alpha_function(reduced_temperature))

    pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)

    rho_V_r = rho_V / rho_c
    rho_L_r = rho_L / rho_c

    gamma_VL_r = integral_VL = integrate_helholtz(eos, reduced_temperature, rho_V_r, rho_L_r, rho_inf=rho_V_r)

    phi_values[reduced_temperature] = np.linspace(-phi_limits[reduced_temperature], phi_limits[reduced_temperature], num=25)

    contact_angles[reduced_temperature] = []

    for phi_r in phi_values[reduced_temperature]:
        contact_angle = calculate_contact_angle(
            phi_r, 
            gamma_VL_r=gamma_VL_r, 
            eos=eos, 
            reduced_temperature=reduced_temperature,
            rho_V_r=rho_V_r,
            rho_L_r=rho_L_r
        )
        contact_angles[reduced_temperature].append(contact_angle)

    contact_angles[reduced_temperature] = np.array(contact_angles[reduced_temperature])

    plt.plot(phi_values[reduced_temperature], 
             contact_angles[reduced_temperature], 
             marker='o', label='Reduced temperature = %.2f' %reduced_temperature)

ax = plt.gca()
ax.grid(True)

ax.set_xlabel(xlabel='Specific Surface Energy')
ax.set_ylabel(ylabel='Contact Angle')
ax.legend(loc='upper right')

plt.show()

In [385]:
print phi_r, reduced_temperature

-0.275 0.8


In [454]:
# Phi limits (so functions do not diverge)
# Tr = 0.9 -> phi_r = 0.35
# Tr = 0.8 -> phi_r = 0.615
# Tr = 0.7 -> phi_r = 0.8
# Tr = 0.65 -> phi_r = 0.225
# n-Butane properties
critical_pressure = 6894.75729 * 550.7
critical_temperature = (5.0 / 9.0) * 765.3 # [K]
acentric_factor = 0.192800 # [-]
molar_mass = 58.12 * 0.001 # [kg/mol]

reduced_temperature = 0.7

temperature = reduced_temperature * critical_temperature
eos = PengRobinsonEos(critical_pressure, critical_temperature, acentric_factor, molar_mass)
rho_c = eos.critical_pressure * eos.molar_mass / (eos.critical_z_factor * GAS_CONST * eos.critical_temperature)

pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)
eos.pressure = eos.pressure.subs(alpha, eos.alpha_function(reduced_temperature))

pressure, rho_V, rho_L = eos.calculate_pressure_and_densities(temperature)

rho_V_r = rho_V / rho_c
rho_L_r = rho_L / rho_c

gamma_VL_r = integral_VL = integrate_helholtz(eos, reduced_temperature, rho_V_r, rho_L_r, rho_inf=rho_V_r)

In [475]:
contact_angle = calculate_contact_angle(
        phi_r=-0.355, 
        gamma_VL_r=gamma_VL_r, 
        eos=eos, 
        reduced_temperature=reduced_temperature,
        rho_V_r=rho_V_r,
        rho_L_r=rho_L_r
    )
print contact_angle

114.046893448503


In [439]:
rho_SV_r, rho_SL_r = calculate_surface_densities(
    phi_r=-0.35, 
    reduced_temperature=0.8, 
    rho_V_r=rho_V_r, 
    rho_L_r=rho_L_r, 
    rho_inf=rho_V_r
    )
print rho_SV_r, rho_SL_r
print rho_V_r, rho_L_r

0.0391992647909 2.47873336441
0.0921619264335243 2.63211907027939


In [368]:
print phi_r

0.225


In [380]:
phi_values[reduced_temperature]

array([-0.238     , -0.21816667, -0.19833333, -0.1785    , -0.15866667,
       -0.13883333, -0.119     , -0.09916667, -0.07933333, -0.0595    ,
       -0.03966667, -0.01983333,  0.        ,  0.01983333,  0.03966667,
        0.0595    ,  0.07933333,  0.09916667,  0.119     ,  0.13883333,
        0.15866667,  0.1785    ,  0.19833333,  0.21816667,  0.238     ])

In [434]:
contact_angles

{0.65: array([103.210628617533, 102.088906568220, 100.972961803070,
        99.8618039933652, 98.7547274255040, 97.6511755114256,
        96.5506906409213, 95.4528952069945, 94.3574907719681,
        93.2642765296204, 92.1732049813091, 91.0845542545362, 90.0,
        88.9277806226994, 87.8579247902963, 86.7876792014217,
        85.7158052346745, 84.6414298992758, 83.5638429467905,
        82.4824209199960, 81.3965901669652, 80.3058057324469,
        79.2095378524827, 78.1072624831875, 76.9984541130239], dtype=object),
 0.7: array([105.871756791953, 104.521756317339, 103.178939922360,
        101.842460384343, 100.511549558268, 99.1855081182527,
        97.8637014149551, 96.5455617902067, 95.2306002197852,
        93.9184355898738, 92.6088665941193, 91.3020854828429, 90.0,
        88.7125929281646, 87.4273528608209, 86.1407013748920,
        84.8509287743651, 83.5567709264079, 82.2571561373558,
        80.9511090472361, 79.6377026158959, 78.3160295603387,
        76.9851829365506, 75.64

In [435]:
phi_values

{0.65: array([ -2.25000000e-01,  -2.06250000e-01,  -1.87500000e-01,
         -1.68750000e-01,  -1.50000000e-01,  -1.31250000e-01,
         -1.12500000e-01,  -9.37500000e-02,  -7.50000000e-02,
         -5.62500000e-02,  -3.75000000e-02,  -1.87500000e-02,
         -2.77555756e-17,   1.87500000e-02,   3.75000000e-02,
          5.62500000e-02,   7.50000000e-02,   9.37500000e-02,
          1.12500000e-01,   1.31250000e-01,   1.50000000e-01,
          1.68750000e-01,   1.87500000e-01,   2.06250000e-01,
          2.25000000e-01]),
 0.7: array([-0.238     , -0.21816667, -0.19833333, -0.1785    , -0.15866667,
        -0.13883333, -0.119     , -0.09916667, -0.07933333, -0.0595    ,
        -0.03966667, -0.01983333,  0.        ,  0.01983333,  0.03966667,
         0.0595    ,  0.07933333,  0.09916667,  0.119     ,  0.13883333,
         0.15866667,  0.1785    ,  0.19833333,  0.21816667,  0.238     ]),
 0.8: array([-0.275     , -0.25208333, -0.22916667, -0.20625   , -0.18333333,
        -0.16041667,

In [428]:
reduced_temperature_values = [0.65, 0.7, 0.8, 0.9]

for reduced_temperature in reduced_temperature_values:
    plt.plot(phi_values_ref[reduced_temperature], 
             contact_angles_ref[reduced_temperature], 
             marker='o', label='Reduced temperature = %.2f' %reduced_temperature)

ax = plt.gca()
ax.grid(True)

ax.set_xlabel(xlabel='Specific Surface Energy')
ax.set_ylabel(ylabel='Contact Angle')
ax.legend(loc='upper right')

plt.show()